# HSC
Markov process with fixed-size population with k-types such that the type 0 is the wild-type with growth rate of `B0`. 

A cells can get a mutation conferring a proliferative advantage upon cell division. We model this process with a Bernouilli trial with success probability of `u`, with units of 1 mutation/division. To compute `u` we can do `u =  MU0 / (B0 * NCELLS)` for the symmetric division case.

For now, all k clones have the same proliferative advantage with k greater than 0.

**Entropy:** based on the code they [developped](https://github.com/emily-mitchell/normal_haematopoiesis/blob/23d221e8d125d78c1e8bcbe05d41d0f3594b0cfb/4_phylogeny_analysis/scripts/shannon_diversity.Rmd#L147), I think they define entropy as in [here](http://math.bu.edu/people/mkon/J6A.pdf) using the phylogenetic tree.
We just compute the entropy from the number of cells: we consider a class being the cells with the same number of mutations and compute the abbundance of those classes, that is the abbundance of cells with the same number of mutations.

## How to use it
Install a version of python greather or equal to 3.11 and then install `seaborn`, `scipy`, `pandas`, `ipykernel` with pip.
Then, install `futils` and `hscpy` in editable mode.
Finally, on the cluster, make this env availbale as a ipython kernel.

In [ ]:
%%bash
cd ../hsc/
git pull
cargo b --release

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import socket
import seaborn as sns
import sys

from pathlib import Path

from hscpy import sfs
from hscpy.figures import burden as burden_figures
from hscpy.figures import sfs as sfs_figures
from hscpy.figures import simulations
from hscpy.figures import variant as variant_figures
from hscpy.figures import PlotOptions

from futils import parse_version

PATH2BIN = Path("~").expanduser() / "hsc/target/release"
assert PATH2BIN.is_dir()

YEARS = 100
YEARS_ENTROPY = 1
RUNS = 64
NB_TIMEPOINTS = 20
DETECTION_THRESH = 0.01
SUBCLONES = 60
USE_SCRATCH = True
mitchell_ages = (0, 29, 38, 48, 63, 75, 81)

SAVE = True
BIGLABELS = False
FIGSIZE = [5, 3] if BIGLABELS else [6.4, 4.8]  # default matplotlib
PDF = True
EXTENSION = ".pdf" if PDF else ".png"
NAIVE = True  # TODO remove
PLOT_OPTIONS = PlotOptions(figsize=FIGSIZE, extension=EXTENSION, save=SAVE)

In [ ]:
if socket.gethostname() == "5X9ZYD3":
    PATH2SIMS = Path("/mnt/c/Users/terenz01/Documents/SwitchDrive/PhD/hsc")
elif socket.gethostname() == "LAPTOP-CEKCHJ4C":
    PATH2SIMS = Path("/mnt/c/Users/fra_t/Documents/PhD/hsc")
else:
    PATH2SIMS = Path("~").expanduser()

PATH2SIMS /= Path("totalVariantFracTime.csv")
assert PATH2SIMS.is_file(), f"cannot find totalVariantFracTime.csv from {PATH2SIMS}"

In [ ]:
%%bash -s "$PATH2BIN" --out version
$1/hsc --version

In [ ]:
VERSION = parse_version(version)
if USE_SCRATCH:
    if NAIVE:
        # PATH2SAVE = Path(f"/data/scratch/hfx923/hsc-draft/naive/{VERSION}")
        # todo fix regression in naive: job arrays do not take the version
        PATH2SAVE = Path(f"/data/scratch/hfx923/hsc-draft/naive/")
    else:
        PATH2SAVE = Path(f"/data/scratch/hfx923/hsc-draft/{VERSION}")
else:
    PATH2SAVE = Path(f"./{VERSION}")

print("Running hsc with version:", VERSION)

## Mitchell's data

In [ ]:
summary = pd.read_csv(PATH2SIMS.parent / "Summary_cut.csv", index_col=0)
summary.cell_type = summary.cell_type.astype("category")
summary.sample_type = summary.sample_type.astype("category")
summary.sort_values(by="age", inplace=True)
summary.reset_index(inplace=True)
ages = summary.age.unique()
closest_age = dict.fromkeys(ages)
# neglect some duplicated colonies e.g. summary.colony_ID == "11_E07"
summary = summary.merge(
    summary[["donor_id", "age"]]
    .groupby("donor_id")
    .count()
    .reset_index()
    .rename(columns={"age": "cells"}),
    on="donor_id",
    validate="many_to_one",
    how="left",
)
summary.dtypes

In [ ]:
print(summary.describe())
print(f"\n\ncell types: \n{summary.cell_type.value_counts()}")
print(f"\n\nsample types: \n{summary.sample_type.value_counts()}")
print(f"\n\ntimepoints: \n{summary.timepoint.value_counts()}")
print(
    f'\n\nages and cells: \n{summary[["donor_id", "cells", "age"]].drop_duplicates()}'
)
print(
    f'\n\nmutations per donor: \n{summary[["donor_id", "number_mutations"]].groupby("donor_id").sum()}'
)

In [ ]:
for i in summary.donor_id.unique():
    fig, ax = plt.subplots(1, 1, tight_layout=True, figsize=(6, 4))
    sns.histplot(
        data=summary[summary.donor_id == i],
        x="number_mutations",
        hue="donor_id",
        kde=True,
        bins=50,
        ax=ax,
        stat="count",
    )
    if SAVE:
        plt.savefig(f"./{i}_burden{EXTENSION}")
    plt.show()

In [ ]:
descr = (
    summary.loc[summary.age == 0, ["donor_id", "number_mutations"]]
    .groupby("donor_id")
    .describe()
)
descr

In [ ]:
descr[("number_mutations", "mean")].mean() / (2 * np.log(200_000 - 2))

In [ ]:
descr[("number_mutations", "std")] ** 2

## Simulations

In [ ]:
SAMPLE_STRONG = int(summary.cells.mean())
SAMPLE_WEAK = SAMPLE_STRONG * 10
NCELLS = 200_000
# mean of the Bernouilli trial (prob of success) to get an asymmetric
# division upon cell division, units are [1 asymmetric division / division]
P_ASYMMETRIC = 0

# division rate for the wild-type in units of [division / (year * cell)]
# Welch, J.S. et al. (2012) ‘The Origin and Evolution of Mutations in Acute Myeloid Leukemia’,
# Cell, 150(2), pp. 264–278
B0 = 1  # TODO: double check this, should be between 2 and 20?

## NEUTRAL RATES with untis [mut/(year * cell)]
#
# Abascal, F. et al. (2021) ‘Somatic mutation landscapes at single-molecule resolution’,
# Nature, 593(7859), pp. 405–410. fig. 2b
# see also fig 1b of Mitchell, E. et al.
# (2022) ‘Clonal dynamics of haematopoiesis across the human lifespan’,
# Nature, 606(7913), pp. 343–350
# MITCHELL's has a mutation rate estimate at 16.8
# we disantagle the mutation rate into a background and division
# such that the sum is 16.8
MU_BACKGROUND = 15.5
MU_DIVISION = 1.3
MU_EXP = 1.3

## FIT CLONES
#
# avg fit mutations arising in 1 year, units are [mutations/year]
# from ABC's inference
MU0 = 2
# proliferative advantage conferred by fit mutations, all clones
# have the same proliferative advantage for now. Units are
# [mutation / division]
S = 0.11
# mean of the Bernouilli trial (prob of success) to get a fit variant upon
# cell division, units are [1 mutation/division]
u = MU0 / (B0 * NCELLS)
# should be 2.0 × 10−3 per HSC per year according to Mitchell, E. et al.
# (2022) ‘Clonal dynamics of haematopoiesis across the human lifespan’,
# Nature, 606(7913), pp. 343–350
# driver mutations enter the HSC compartment at 2.0 × 10−3 per HSC per year
print(f"average sucess rate of occurence of 1 fit mutation upon cell division u={u}")

### Positive selection

#### Rust simulations

We run the simulations with and without subsampling at the same time.

In [ ]:
sim_options_population = simulations.SimulationOptions(
    runs=RUNS,
    cells=NCELLS,
    sample=NCELLS,
    path2save=PATH2SAVE,
    neutral_rate=MU_BACKGROUND,
    nb_timepoints=NB_TIMEPOINTS,
    last_timepoint_years=YEARS,
    nb_subclones=SUBCLONES,
    s=S,
)

sim_options_subsampling_strong = simulations.SimulationOptions(
    runs=RUNS,
    cells=NCELLS,
    sample=SAMPLE_STRONG,
    path2save=PATH2SAVE,
    neutral_rate=MU_BACKGROUND,
    nb_timepoints=NB_TIMEPOINTS,
    last_timepoint_years=YEARS,
    nb_subclones=SUBCLONES,
    s=S,
)

sim_options_subsampling_weak = simulations.SimulationOptions(
    runs=RUNS,
    cells=NCELLS,
    sample=SAMPLE_WEAK,
    path2save=PATH2SAVE,
    neutral_rate=MU_BACKGROUND,
    nb_timepoints=NB_TIMEPOINTS,
    last_timepoint_years=YEARS,
    nb_subclones=SUBCLONES,
    s=S,
)

In [ ]:
%%bash -s "$PATH2BIN" "$sim_options_population.path2save" "$B0" "$MU0" "$sim_options_population.neutral_rate" "$sim_options_population.s" "$P_ASYMMETRIC" "$sim_options_population.runs" "$sim_options_population.cells" "$YEARS" "$sim_options_population.nb_timepoints" "$sim_options_subsampling_strong.sample" "$sim_options_subsampling_weak.sample" "$YEARS_ENTROPY"
# rm -rf $2
# $1/hsc -c $9 -y ${10} -r $8 --b0 $3 --mu0 $4 --neutral-rate $5 --p-asymmetric $7 --snapshot-entropy ${14} --subsample ${12} ${13} --snapshots ${11} --mean-std 0.1 0.03 --exponential $2

In [ ]:
for i, f in enumerate((PATH2SAVE / "rates/").iterdir()):
    fig, ax = plt.subplots(1, 1)
    pd.read_csv(f, header=None).squeeze().plot(kind="hist", ax=ax, bins=15)
    ax.set_xlim(0.95, 1.25)  # TODO?
    ax.set_title(f"simulation id: {i}")
    plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, tight_layout=True, figsize=FIGSIZE)
sns.histplot(
    data=summary,
    x="number_mutations",
    hue="donor_id",
    kde=True,
    binwidth=10,
    ax=ax,
    stat="percent",
)
sns.move_legend(ax, bbox_to_anchor=(1.01, 1), loc="upper left", frameon=False)
if SAVE:
    plt.savefig(f"./mitchell_burden{EXTENSION}")
plt.show()

In [ ]:
%%time
# compute the correction for the SFS with sampled distributions from
# https://www.biorxiv.org/content/10.1101/2022.11.07.515470v2
corrected_variants_one_over_1_squared = dict()
for donor in summary[["donor_id", "age", "cells"]].drop_duplicates().itertuples():
    print(
        f"apply sampling correction to SFS of donor {donor.donor_id} with age {donor.age}"
    )
    corrected_variants_one_over_1_squared[donor.donor_id] = sfs.compute_variants(
        sfs.Correction.ONE_OVER_F_SQUARED,
        pop_size=sim_options_subsampling_strong.cells,
        sample_size=donor.cells,
    )

#### no subsampling

In [ ]:
sims_pop = simulations.Simulations(summary, sim_options_population)

In [ ]:
sfs_figures.plot_simulations_donors_sfs(
    sims_pop,
    PLOT_OPTIONS,
    PATH2SIMS.parent,
    True,
    corrected_variants_one_over_1_squared,
)

In [ ]:
burden_figures.plot_simulations_burden(sims_pop, PLOT_OPTIONS, summary)

In [ ]:
variant_figures.show_variant_plots(
    sims_pop.sim_options, PLOT_OPTIONS, PATH2SIMS, DETECTION_THRESH
)

#### weak subsampling

In [ ]:
sims_weak = simulations.Simulations(summary, sim_options_subsampling_weak)

In [ ]:
sfs_figures.plot_simulations_donors_sfs(
    sims_weak,
    PLOT_OPTIONS,
    PATH2SIMS.parent,
    True,
    corrected_variants_one_over_1_squared,
)

In [ ]:
burden_figures.plot_simulations_burden(sims_weak, PLOT_OPTIONS, summary)

#### strong subsampling

In [ ]:
sims_strong = simulations.Simulations(summary, sim_options_subsampling_strong)

In [ ]:
sfs_figures.plot_simulations_donors_sfs(
    sims_strong,
    PLOT_OPTIONS,
    PATH2SIMS.parent,
    True,
    corrected_variants_one_over_1_squared,
)

In [ ]:
burden_figures.plot_simulations_burden(sims_strong, PLOT_OPTIONS, summary)

In [ ]:
# TODO 1 clone logistic fn

### Neutral scenario

In [ ]:
assert False

In [ ]:
%%bash -s "$PATH2BIN" "$sim_options_subsampling_strong.path2save" "$B0" "$MU0" "$sim_options_population.neutral_rate" "$sim_options_population.s" "$P_ASYMMETRIC" "$sim_options_population.runs" "$sim_options_population.cells" "$YEARS" "$sim_options_population.nb_timepoints" "$sim_options_subsampling_strong.sample" "$sim_options_subsampling_weak.sample" "$YEARS_ENTROPY"
# rm -rf $2
# $1/hsc -c $9 -y ${10} -r $8 --b0 $3 --mu0 $4 --neutral-rate $5 --p-asymmetric $7 --snapshot-entropy ${14} --subsample ${12} ${13} --snapshots ${11} --neutral --exponential $2

In [ ]:
sfs_figures.show_entropy_plots(
    sim_options_population, PLOT_OPTIONS, mitchell_ages, early_variants_only=True
)

sfs_figures.show_entropy_plots(
    sim_options_population, PLOT_OPTIONS, mitchell_ages, early_variants_only=False
)

In [ ]:
sfs_figures.show_entropy_plots(
    sim_options_subsampling_weak, PLOT_OPTIONS, mitchell_ages, early_variants_only=True
)

sfs_figures.show_entropy_plots(
    sim_options_subsampling_weak, PLOT_OPTIONS, mitchell_ages, early_variants_only=False
)

In [ ]:
sfs_figures.show_entropy_plots(
    sim_options_subsampling_strong,
    PLOT_OPTIONS,
    mitchell_ages,
    early_variants_only=True,
)

sfs_figures.show_entropy_plots(
    sim_options_subsampling_strong,
    PLOT_OPTIONS,
    mitchell_ages,
    early_variants_only=False,
)